In [34]:
import h5py
import json, csv
import os
from pathlib import Path
import math
import pandas as pd
import numpy as np

import torch 
from transformers import AutoTokenizer
from elasticsearch_dsl import Document, Integer, Text, DenseVector
from elasticsearch_dsl.connections import connections

import time
import requests
from tqdm import tqdm
from dataclasses import dataclass

from dataclasses_json import dataclass_json
from dacite import from_dict
from nltk import tokenize
from omegaconf import OmegaConf 
from pprint import pprint

from transformers import DistilBertTokenizer
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence, pack_padded_sequence

import src.data
from src.data import load_metadata, find_paths, relative_file_path

import sys
sys.path.append('../contrastive_mm2/') 
from gru_test2 import mmModule, Cfg
from prepare_index_sentencelevel2 import read_metadata_subset

# Load static configuration variables. 
config_path = "./config.yaml"
conf = OmegaConf.load(config_path)
print("[cudacheck] Is cuda available? ", torch.cuda.is_available())

[cudacheck] Is cuda available?  False


In [35]:
"""TO ARGS"""
model_path = "E:\msc_thesis\code\contrastive_mm2\logs\lisa_v2-simcse_loss_rnn_relu_768_2e-05_2022-05-17_06-58-44"
model_path = '/Users/casper/Documents/UvAmaster/b23456_thesis/msc_thesis/code/contrastive_mm2/logs/windows_gru2-clip_loss_gru_gelu_768_5e-05_2022-05-26_21-51-25'
model_path = '/Users/casper/Documents/UvAmaster/b23456_thesis/msc_thesis/code/contrastive_mm2/logs/windows_gru2-clip_loss_followup'




transcripts_path = '/Users/casper/Documents/UvAmaster/b23456_thesis/msc_thesis/code/data/sp/podcasts-no-audio-13GB/podcasts-transcripts'
# transcripts_path = 'E:/msc_thesis/code/data/sp/podcasts-no-audio-13GB/podcasts-transcripts'

# Whether we create for the train or the tes split. 
traintest = 'test'


In [36]:

# Creating output directory.
save_name = os.path.split(model_path)[-1]
save_path = os.path.join(conf.yamnet_topic_embed_path, save_name + "_" + traintest)
Path(save_path).mkdir(parents=True, exist_ok=True)

# Reading the metadata.
metadata_testset, topics_df, topics_df_targets = read_metadata_subset(conf, traintest='test')

# Remove duplicate rows
metadata_testset = metadata_testset.drop_duplicates(subset=['episode_filename_prefix']).sort_values('episode_filename_prefix')
print("[main] Topic test set loaded: ", len(metadata_testset))


[main] metadata loaded  105360
[main] topics_test loaded  50
[main] topics_df_targets loaded for test set 9426
[main] metadata_subset loaded  9426
[main] Topic test set loaded:  3421


In [37]:

# Loading the model.
print("[Load model] from ", model_path)
model_weights = os.path.join(model_path, "output/full_model_weights.pth")
print("[Load model] weights loaded")
model_config_path = os.path.join(model_path, 'config.json')

# Opening JSON file
f = open(model_config_path)
model_config = json.load(f)
print("using config: , ", model_config)
print("\n\n")
CFG = from_dict(data_class=Cfg, data=model_config)
CFG.device = "cuda" if torch.cuda.is_available() else "cpu"
# mutual_embed_dim = CFG.final_projection_dim
print("[Load model] config loaded: ", CFG)

[Load model] from  /Users/casper/Documents/UvAmaster/b23456_thesis/msc_thesis/code/contrastive_mm2/logs/windows_gru2-clip_loss_followup
[Load model] weights loaded
using config: ,  {'audio_dropout': 0.1, 'audio_encoder_input': 1024, 'audio_hidden_dim': 768, 'audio_layer_dim': 2, 'device': 'cuda', 'lr': 5e-05, 'mutual_embedding_dim': 768, 'text_dropout': 0.1, 'text_max_length': 32, 'text_model_name': 'distilbert-base-uncased', 'text_tokenizer': 'distilbert-base-uncased', 'weight_decay': 0.01, 'audio_activation': 'gelu', 'audio_proj_head': 'gru', 'batch_size': 128, 'cuda_number': '0', 'dev_dataset': 'sts', 'eval_every': 100, 'final_projection_dim': 768, 'fp16': False, 'load_checkpoint': False, 'load_checkpoint_path': './logs/load_test2/output/checkpoint.pth', 'load_model': False, 'load_model_path': './logs/load_test2/output/full_model_weights.pth', 'log_dir': './logs', 'loss_type': 'clip_loss', 'no_cuda': False, 'normalize': True, 'num_epochs': 10, 'pad_pack': True, 'save_checkpoint': Tr

In [38]:
# Load model and tokenizer
full_model = mmModule(CFG)
full_model.load_state_dict(torch.load(model_weights,  map_location=CFG.device))                      
full_model.eval()

tokenizer = AutoTokenizer.from_pretrained(CFG.text_model_name)
# tokenizer = DistilBertTokenizer.from_pretrained(CFG.text_tokenizer)
print("[Load model] model loaded: ", model_weights)
print(full_model.state_dict()['audio_model.0.seq_model.weight_ih_l1'][0][0])

[Transformer_multimodal] Configurations used: 
 DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.16.2",
  "vocab_size": 30522
}



Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[Load model] model loaded:  /Users/casper/Documents/UvAmaster/b23456_thesis/msc_thesis/code/contrastive_mm2/logs/windows_gru2-clip_loss_followup/output/full_model_weights.pth
tensor(0.0674)


In [39]:

%load_ext autoreload
%autoreload 2

from gru_test2 import MMloader

# Create dataloader of test set
data_loader = MMloader(CFG)
train_loader = data_loader.train_loader
val_loader = data_loader.val_loader
test_loader = data_loader.test_loader # Not used!

# TODO: move this into __init__ of dataloader
data_loader.test_dataset.tokenizer = tokenizer
data_loader.test_dataset.text_max_length = CFG.text_max_length




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[MMloader] train dataset  sp
[spDataset] init from directory  ../data/sp/yamnet/processed/ train
[spDataset] found 1 h5py files
[del] h5py_file:  ../data/sp/yamnet/processed/train/0_0_embeds.h5
[MMloader] train dataset loaded, length:  19344
[spDataset] init from directory  ../data/sp/yamnet/processed/ val
[spDataset] found 1 h5py files
[del] h5py_file:  ../data/sp/yamnet/processed/val/0_0_embeds.h5
[spDataset] init from directory  ../data/sp/yamnet/processed/ test
[spDataset] found 97 h5py files
[del] h5py_file:  ../data/sp/yamnet/processed/test/0_N_embeds.h5
[del] h5py_file:  ../data/sp/yamnet/processed/test/1_R_embeds.h5
[del] h5py_file:  ../data/sp/yamnet/processed/test/2_1_embeds.h5
[del] h5py_file:  ../data/sp/yamnet/processed/test/1_G_embeds.h5
[del] h5py_file:  ../data/sp/yamnet/processed/test/2_M_embeds.h5
[del] h5py_file:  ../data/sp/yamnet/processed/test/0_2_embeds.h5
[del] h5py_file:  ..

## Standard validation

In [ ]:
iterator = iter(test_loader)
max_steps = len(test_loader) 
print(max_steps)


for step in range(max_steps):
    print("step {}/{}".format(step, max_steps))
    batch = next(iterator)
    
    (tok_sentences, audio_features, seq_len, targets) = batch

    with torch.no_grad():
        reps_sentences = full_model.text_model(tok_sentences)['sentence_embedding']
        reps_audio = full_model.audio_model((audio_features, seq_len))
        
        audio_logits =  (reps_audio @ reps_sentences.t()) * CFG.scale
        text_logits = audio_logits.t()
        e:
        audio_logits = audio_logits / audio_logits.norm(dim=1, keepdim=True)
        text_logits = text_logits / text_logits.norm(dim=1, keepdim=True)
        
        probs = audio_logits.softmax(dim=-1).cpu().numpy()


4105
step 0/4105
step 1/4105
step 2/4105
step 3/4105
step 4/4105
step 5/4105
step 6/4105
step 7/4105
step 8/4105
step 9/4105
step 10/4105
step 11/4105
step 12/4105
step 13/4105
step 14/4105
step 15/4105
step 16/4105
step 17/4105
step 18/4105
step 19/4105
step 20/4105
step 21/4105
step 22/4105
step 23/4105
step 24/4105
step 25/4105
step 26/4105
step 27/4105
step 28/4105
step 29/4105
step 30/4105
step 31/4105
step 32/4105
step 33/4105
step 34/4105
step 35/4105
step 36/4105
step 37/4105
step 38/4105
step 39/4105
step 40/4105
step 41/4105
step 42/4105
step 43/4105
step 44/4105
step 45/4105
step 46/4105
step 47/4105
step 48/4105
step 49/4105
step 50/4105
step 51/4105
step 52/4105
step 53/4105
step 54/4105
step 55/4105
step 56/4105
step 57/4105
step 58/4105
step 59/4105
step 60/4105
step 61/4105
step 62/4105
step 63/4105
step 64/4105
step 65/4105
step 66/4105
step 67/4105


In [26]:
probs = audio_logits.softmax(dim=-1).cpu().numpy()
# print(probs.argmax(axis=0))
ground_truth = torch.arange(128)
# print(ground_truth)


print("accuracy: ")
acc = torch.eq(torch.tensor(probs.argmax(axis=0)), ground_truth).sum() / ground_truth.shape[0]
print(acc)

accuracy: 
tensor(0.3672)


## Create embeddings of shows in test set

In [27]:
processed_text = []
processed_audio = []

topic_norm_reps_text = []
topic_norm_reps_audio = []

matrix_targets = []
iterator = iter(test_loader)
for step in range(2):
    batch = next(iterator)
    
    (tok_sentences, audio_features, seq_len, targets) = batch

    with torch.no_grad():
        # print("[del] get embeds: ")
        reps_sentences = full_model.text_model(tok_sentences)['sentence_embedding']
        # embeds = self.model.text_encoder(input_ids=input_ids, attention_mask=attention_mask)
        #reps_sentences = reps_sentences.detach().cpu().numpy()
        
        
        reps_audio = full_model.audio_model((audio_features, seq_len))
        
        # Normalized representations
        topic_norm_reps_text.append(reps_sentences / reps_sentences.norm(dim=1, keepdim=True))
        topic_norm_reps_audio.append(reps_audio / reps_audio.norm(dim=1, keepdim=True))
        
        #print(reps_audio.shape)
        audio_logits =  (reps_audio @ reps_sentences.t()) * CFG.scale
        text_logits = audio_logits.t()
        
        matrix_targets.extend(targets)
        
        
#         audio_logits = audio_logits / audio_logits.norm(dim=1, keepdim=True)
#         text_logits = text_logits / text_logits.norm(dim=1, keepdim=True)
        
#         #probs = audio_logits.softmax(dim=-1).cpu().numpy()
        
#         processed_text.append(audio_logits)
        
#         processed_audio.append(audio_logits)
        
#     print(probs)
print()

[cudacheck] Is cuda available?  False
torch.Size([128, 768])
torch.Size([128, 768])



In [28]:
print(topic_norm_reps_audio[0].shape)

topic_norm_reps_audio = torch.cat(topic_norm_reps_audio, dim=0)
print(topic_norm_reps_audio.shape)

topic_norm_reps_text = torch.cat(topic_norm_reps_text, dim=0)
print(topic_norm_reps_text.shape)

# probs = audio.softmax(dim=-1).cpu().numpy()
# print(probs)

torch.Size([128, 768])
torch.Size([256, 768])
torch.Size([256, 768])


In [30]:
# similarity = text.cpu().numpy().T @  audio.cpu().numpy()


# print(matrix_targets)
# topics_df

In [32]:
# result = similarity.argmax(axis=0)
# ground_truth = torch.arange(similarity.shape[0])

# stats = torch.eq(torch.tensor(result), ground_truth).sum()
# print(stats)

In [33]:
### Validation DF
ids_we_can_use = [m.split('_')[0] for m in matrix_targets]
val_df = topics_df_targets[topics_df_targets['episode_uri'].isin(ids_we_can_use)]

### TODO: dit naar boven
positive_episodes = val_df.loc[val_df['bin_score'] == 1].copy()
positive_eplist = positive_episodes['episode_uri'].tolist()


for i, row in val_df.iterrows():
    ifor_val = 0
    if row['episode_uri'] in positive_eplist:
        #print("IN TESTLIST")
        ifor_val = 1
    val_df.loc[i,'ep_score'] = ifor_val
    
val_df.ep_score = val_df.ep_score.astype(int)
val_df.head(50)

/opt/miniconda3/envs/troep/lib/python3.8/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/opt/miniconda3/envs/troep/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/opt/miniconda3/envs/troep/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

,num,unk,episode_uri_time,score,bin_score,episode_uri,time,ep_score
4767,34,0,75cpxDXpVz9ScTVNIWz5d6_420,0,0,75cpxDXpVz9ScTVNIWz5d6,420,0
7540,13,0,3QGmWCiVOsKzY85lnSglF6_1980,0,0,3QGmWCiVOsKzY85lnSglF6,1980,0
7541,13,0,3QGmWCiVOsKzY85lnSglF6_2040,0,0,3QGmWCiVOsKzY85lnSglF6,2040,0
7542,13,0,3QGmWCiVOsKzY85lnSglF6_2100,0,0,3QGmWCiVOsKzY85lnSglF6,2100,0
7543,13,0,3QGmWCiVOsKzY85lnSglF6_2160,0,0,3QGmWCiVOsKzY85lnSglF6,2160,0


In [180]:
# Create text embeddings of queries
query_field = 'query'
query_nr = 0
query = topics_df[query_field].iloc[query_nr]


queries = topics_df[query_field][:45].tolist()
tokenized_queries = tokenizer(
    queries, padding=True, truncation=True, max_length=32, return_tensors='pt', return_token_type_ids=True,
)


print(tokenized_queries)

query_yamnets = []
query_lengths = []
for idx, row in topics_df[:45].iterrows():
    query_num = row.num
    print(query_num)
    query_embed_path  = os.path.join(conf.yamnet_query_embed_path, str(query_num) + ".h5")
    inbetween_yamnet_embeds = pd.read_hdf(query_embed_path)
    print(type(inbetween_yamnet_embeds))
    print("LEN: ", len(inbetween_yamnet_embeds))
    query_lengths.append(len(inbetween_yamnet_embeds))
    tmp = torch.Tensor(inbetween_yamnet_embeds.values)
    print(tmp.shape)
    query_yamnets.append(tmp)
    
    
    
    #lengths.append(len(example[1]))

print(query_yamnets)

padded_query_yamnets = pad_sequence(query_yamnets, batch_first=True)




{'input_ids': tensor([[  101,  8398,  2655,  5969,  2343,   102,     0,     0],
        [  101, 10321, 22061,  5823,  5320,   102,     0,     0],
        [  101,  2129,  2000,  5660,  4977,   102,     0,     0],
        [  101, 10047,  5521,  4967,  2476,   102,     0,     0],
        [  101,  4319, 13449,  7233,   102,     0,     0,     0],
        [  101,  2379,  2331,  6322,   102,     0,     0,     0],
        [  101, 16110,  2055, 16110,  2015,   102,     0,     0],
        [  101,  5320,  1998,  9740,  1997,  3748, 26332,   102],
        [  101,  2051,  2090,  6295,   102,     0,     0,     0],
        [  101,  2308,  1999,  7872,   102,     0,     0,     0],
        [  101,  9932,  1999,  9871,   102,     0,     0,     0],
        [  101,  3465,  1997,  2775, 16302,   102,     0,     0],
        [  101,  2238, 17389,  3372,  2232,   102,     0,     0],
        [  101, 24188, 25083,  8516, 14633,   102,     0,     0],
        [  101, 10289,  8214,  2543,   102,     0,     0,     

<class 'pandas.core.frame.DataFrame'>
LEN:  4
torch.Size([4, 1024])
42
<class 'pandas.core.frame.DataFrame'>
LEN:  3
torch.Size([3, 1024])
43
<class 'pandas.core.frame.DataFrame'>
LEN:  4
torch.Size([4, 1024])
44
<class 'pandas.core.frame.DataFrame'>
LEN:  4
torch.Size([4, 1024])
45
<class 'pandas.core.frame.DataFrame'>
LEN:  4
torch.Size([4, 1024])
46
<class 'pandas.core.frame.DataFrame'>
LEN:  4
torch.Size([4, 1024])
47
<class 'pandas.core.frame.DataFrame'>
LEN:  5
torch.Size([5, 1024])
48
<class 'pandas.core.frame.DataFrame'>
LEN:  4
torch.Size([4, 1024])
49
<class 'pandas.core.frame.DataFrame'>
LEN:  3
torch.Size([3, 1024])
50
<class 'pandas.core.frame.DataFrame'>
LEN:  4
torch.Size([4, 1024])
51
<class 'pandas.core.frame.DataFrame'>
LEN:  3
torch.Size([3, 1024])
52
<class 'pandas.core.frame.DataFrame'>
LEN:  3
torch.Size([3, 1024])
53
<class 'pandas.core.frame.DataFrame'>
LEN:  6
torch.Size([6, 1024])
[tensor([[0.0000, 0.2856, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0014

In [181]:
query_text = []

query_audio = []
query_field = 'query'

with torch.no_grad():


    # print("[del] get embeds: ")
    reps_sentences = full_model.text_model(tokenized_queries)['sentence_embedding']
    # embeds = self.model.text_encoder(input_ids=input_ids, attention_mask=attention_mask)
    #reps_sentences = reps_sentences.detach().cpu().numpy()


    reps_audio = full_model.audio_model((padded_query_yamnets, query_lengths))


    audio_logits =  (reps_audio @ reps_sentences.t()) * CFG.scale
    text_logits = audio_logits.t()


    audio_logits = audio_logits / audio_logits.norm(dim=1, keepdim=True)
    text_logits = text_logits / text_logits.norm(dim=1, keepdim=True)

    #probs = audio_logits.softmax(dim=-1).cpu().numpy()

    query_text_logits = text_logits
    query_audio_logits = audio_logits
    
    query_norm_reps_audio = reps_audio / reps_audio.norm(dim=1, keepdim=True)
    query_norm_reps_text = reps_sentences / reps_sentences.norm(dim=1, keepdim=True)
#     print(probs)



In [182]:
print(query_text_logits)
print(query_text_logits.shape)


print(query_audio_logits)
print(query_audio_logits.shape)

tensor([[ 0.2221, -0.0591, -0.0153,  ...,  0.0341, -0.2017, -0.0945],
        [ 0.3498,  0.0355,  0.0704,  ...,  0.0071,  0.1612, -0.0263],
        [-0.1308,  0.0089,  0.1843,  ...,  0.0023,  0.2269,  0.0049],
        ...,
        [-0.1502,  0.0006,  0.0308,  ...,  0.2366,  0.0736,  0.1026],
        [-0.0960,  0.0699,  0.1117,  ..., -0.0189,  0.1210,  0.0348],
        [ 0.2057,  0.1482,  0.0596,  ..., -0.0126,  0.1532,  0.0316]])
torch.Size([45, 45])
tensor([[ 0.0934,  0.3085, -0.0729,  ..., -0.0587, -0.0434,  0.2053],
        [-0.0784,  0.0988,  0.0156,  ...,  0.0007,  0.0998,  0.4668],
        [-0.0064,  0.0618,  0.1023,  ...,  0.0120,  0.0503,  0.0592],
        ...,
        [ 0.0472,  0.0205,  0.0043,  ...,  0.3046, -0.0281, -0.0415],
        [-0.0378,  0.0634,  0.0564,  ...,  0.0128,  0.0244,  0.0682],
        [-0.1632, -0.0952,  0.0112,  ...,  0.1646,  0.0646,  0.1292]])
torch.Size([45, 45])


In [183]:
print(query_norm_reps_audio.shape)
print(query_norm_reps_text.shape)

torch.Size([45, 768])
torch.Size([45, 768])


In [191]:
from torch import topk
k = 50

pred_episodes = {}

similarity = 100 * query_norm_reps_text @ topic_norm_reps_text.T
# print(similarity)
# print(similarity.shape)

probs = similarity.softmax(dim=-1).cpu()
#print(probs.argmax(axis=1))


top_probs, top_labels = probs.topk(k, dim=-1)
# ground_truth = torch.arange(128)
# print(ground_truth)

for num in range(45):
    query = topics_df[query_field][num]
    
    pred_ind = top_labels[num].tolist()
    #print("pred: ", pred_ind)
    
    #pred_targs = matrix_targets[pred_ind]
    pred_epis = [matrix_targets[val].split('_')[0] for val in pred_ind]

    
    tmp = val_df[(val_df.num == num) & (val_df.ep_score==1)]
    tmp = tmp['episode_uri'].tolist()
    num_episodes_relevant = len(set(tmp))
    
    ep_scores = []
    for episode_uri in pred_epis:
        ep_score_row = val_df.loc[val_df['episode_uri'] == episode_uri]
        if len(ep_score_row) > 0 and ep_score_row.num.values[0] == num:
            ep_scores.append(1)
        else:
            ep_scores.append(0)
    
    pred_episodes[query] = {}
    pred_episodes[query]['episodes'] = [matrix_targets[val].split('_')[0] for val in pred_ind]
    pred_episodes[query]['ep_score'] = ep_scores
    
    pred_episodes[query]['prec@3'] = precision_at_k(ep_scores, 3)
    pred_episodes[query]['prec@10'] = precision_at_k(ep_scores, 10)
    pred_episodes[query]['prec@30'] = precision_at_k(ep_scores, 30)
    

    targets = [[1] * num_episodes_relevant + [0] * (len(ep_scores) - num_episodes_relevant)]
    ndcg_ep_score = ndcg_score(targets, [ep_scores], k=30)
    pred_episodes[query]['ndc'] = ndcg_ep_score
    print("done query {}, p@10 {}, ndcg: {}".format(num, pred_episodes[query]['prec@10'], ndcg_ep_score))




# text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
# top_probs, top_labels = text_probs.cpu().topk(5, dim=-1)
# bels = text_probs.cpu().topk(5, dim=-1)



done query 0, p@10 0.0, ndcg: 0.0
done query 1, p@10 0.0, ndcg: 0.0
done query 2, p@10 0.0, ndcg: 0.0
done query 3, p@10 0.0, ndcg: 0.0
done query 4, p@10 0.0, ndcg: 0.0
done query 5, p@10 0.0, ndcg: 0.0
done query 6, p@10 0.0, ndcg: 0.0
done query 7, p@10 0.0, ndcg: 0.0
done query 8, p@10 0.0, ndcg: 0.0
done query 9, p@10 0.0, ndcg: 0.0
done query 10, p@10 0.0, ndcg: 0.0
done query 11, p@10 0.0, ndcg: 0.0
done query 12, p@10 0.0, ndcg: 0.0
done query 13, p@10 0.3, ndcg: 0.0
done query 14, p@10 0.0, ndcg: 0.0
done query 15, p@10 0.0, ndcg: 0.0
done query 16, p@10 0.0, ndcg: 0.0
done query 17, p@10 0.0, ndcg: 0.0
done query 18, p@10 0.0, ndcg: 0.0
done query 19, p@10 0.0, ndcg: 0.0
done query 20, p@10 0.0, ndcg: 0.0
done query 21, p@10 0.0, ndcg: 0.0
done query 22, p@10 0.0, ndcg: 0.0
done query 23, p@10 0.0, ndcg: 0.0
done query 24, p@10 0.0, ndcg: 0.0
done query 25, p@10 0.0, ndcg: 0.0
done query 26, p@10 0.0, ndcg: 0.0
done query 27, p@10 0.0, ndcg: 0.0
done query 28, p@10 0.0, ndcg:

In [141]:
pred_episodes

{'trump call ukrainian president': ['75cpxDXpVz9ScTVNIWz5d6',
  '20Rxox5JAR894PcIMGpsV9',
  '75cpxDXpVz9ScTVNIWz5d6',
  '3QGmWCiVOsKzY85lnSglF6',
  '20Rxox5JAR894PcIMGpsV9',
  '20Rxox5JAR894PcIMGpsV9',
  '3QGmWCiVOsKzY85lnSglF6',
  '3QGmWCiVOsKzY85lnSglF6',
  '20Rxox5JAR894PcIMGpsV9',
  '75cpxDXpVz9ScTVNIWz5d6',
  '20Rxox5JAR894PcIMGpsV9',
  '3QGmWCiVOsKzY85lnSglF6',
  '3QGmWCiVOsKzY85lnSglF6',
  '3QGmWCiVOsKzY85lnSglF6',
  '20Rxox5JAR894PcIMGpsV9',
  '20Rxox5JAR894PcIMGpsV9',
  '3QGmWCiVOsKzY85lnSglF6',
  '3QGmWCiVOsKzY85lnSglF6',
  '75cpxDXpVz9ScTVNIWz5d6',
  '3QGmWCiVOsKzY85lnSglF6',
  '75cpxDXpVz9ScTVNIWz5d6',
  '75cpxDXpVz9ScTVNIWz5d6',
  '75cpxDXpVz9ScTVNIWz5d6',
  '3QGmWCiVOsKzY85lnSglF6',
  '3QGmWCiVOsKzY85lnSglF6',
  '75cpxDXpVz9ScTVNIWz5d6',
  '75cpxDXpVz9ScTVNIWz5d6',
  '3QGmWCiVOsKzY85lnSglF6',
  '3QGmWCiVOsKzY85lnSglF6',
  '3QGmWCiVOsKzY85lnSglF6',
  '3QGmWCiVOsKzY85lnSglF6',
  '3QGmWCiVOsKzY85lnSglF6',
  '3QGmWCiVOsKzY85lnSglF6',
  '20Rxox5JAR894PcIMGpsV9',
  '3QGmWCiVOsK

In [158]:
val_df.head(3)

,num,unk,episode_uri_time,score,bin_score,episode_uri,time,ep_score
4767,34,0,75cpxDXpVz9ScTVNIWz5d6_420,0,0,75cpxDXpVz9ScTVNIWz5d6,420,0
5464,35,0,20Rxox5JAR894PcIMGpsV9_1020,3,1,20Rxox5JAR894PcIMGpsV9,1020,1
5465,35,0,20Rxox5JAR894PcIMGpsV9_1380,3,1,20Rxox5JAR894PcIMGpsV9,1380,1


In [152]:
from sklearn.metrics import ndcg_score

def precision_at_k(predicted, k):
    if k == 0:
        print("precision from k ===0?")
        raise
    pred = (predicted[:k])
    relevant = np.sum(pred)
    num_recommended = float(k)
    p_at_k = relevant / num_recommended
    return p_at_k

# Recall@k = (# of recommended items @k that are relevant) / (total # of relevant items)
def recall_at_k(predicted, num_relevant, k):
    if num_relevant == 0:
        return None
    pred = (predicted[:k])
    relevant = np.sum(pred)
    r_at_k = relevant / num_relevant

    return r_at_k


scores = [[1,1,1,0,1]]
targets = [[1,1,1,1,0]]

print(precision_at_k(scores[0], 5))
print(recall_at_k(scores[0], 0, 2))
ndcg = ndcg_score(targets, scores, k=3)
print(ndcg)
# print(es_url)

0.8
None
0.75


In [157]:

# get score from val df

3QGmWCiVOsKzY85lnSglF6


/opt/miniconda3/envs/troep/lib/python3.8/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/opt/miniconda3/envs/troep/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/opt/miniconda3/envs/troep/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

,num,unk,episode_uri_time,score,bin_score,episode_uri,time,ep_score
4767,34,0,75cpxDXpVz9ScTVNIWz5d6_420,0,0,75cpxDXpVz9ScTVNIWz5d6,420,0
5464,35,0,20Rxox5JAR894PcIMGpsV9_1020,3,1,20Rxox5JAR894PcIMGpsV9,1020,1
5465,35,0,20Rxox5JAR894PcIMGpsV9_1380,3,1,20Rxox5JAR894PcIMGpsV9,1380,1
7540,13,0,3QGmWCiVOsKzY85lnSglF6_1980,0,0,3QGmWCiVOsKzY85lnSglF6,1980,0
7541,13,0,3QGmWCiVOsKzY85lnSglF6_2040,0,0,3QGmWCiVOsKzY85lnSglF6,2040,0
7542,13,0,3QGmWCiVOsKzY85lnSglF6_2100,0,0,3QGmWCiVOsKzY85lnSglF6,2100,0
7543,13,0,3QGmWCiVOsKzY85lnSglF6_2160,0,0,3QGmWCiVOsKzY85lnSglF6,2160,0
